# The Battle of Neighborhoods
#### This notebook will be mainly used to help visitors to NewYork find coffee shops around their hotel or place of staying

##### Firstly: We import the libraries we'll use in this notebook

In [7]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json, requests
import folium

##### Secondly: We make a request to FourSquare to get the data we need which is the coffee shops around a specific area with a specific radius

In [14]:
CLIENT_ID = 'WONHP2ODPL0PPDAGCC1EKCGYHHGHSYSSFLKVKWOUWLZ01CVI'
CLIENT_SECRET = 'GCJ1SL111MWTQCVGQ0HMP004Y4PXK1GSOLAKSLRDWHQQBDMC'
VERSION = '20180323'
latitude = '40.7243'
longitude = '-74.0018'
radius = 500
URL = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query=coffee&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius)
response = requests.get(url=URL).json()
venues = response['response']['venues']
csv_file = pd.json_normalize(venues)

##### Thirdly: We clean the data we got from FourSquare and make it an easy to understand database (includes dropping the NAN cells)

In [15]:
csv_file.drop(columns=['categories', 'referralId', 'location.crossStreet', 'id', 'verified', 'venueChains',
                       'hasPerk', 'location.crossStreet', 'stats.visitsCount', 'beenHere.count',
                       'beenHere.lastCheckinExpiredAt', 'beenHere.marked', 'beenHere.unconfirmedCount',
                       'hereNow.count', 'hereNow.summary', 'hereNow.groups', 'venuePage.id', 'location.neighborhood',
                       'location.labeledLatLngs', 'location.distance', 'location.postalCode', 'location.cc',
                       'location.state', 'location.formattedAddress', 'stats.tipCount', 'stats.usersCount', 
                       'stats.usersCount', 'stats.checkinsCount'], inplace=True)
csv_file.rename(columns={'name':'Name',
                         'location.address':'Address',
                         'location.lat':'Latitude',
                         'location.lng':'Longitude',
                         'location.distance':'Distance',
                         'location.city':'City',
                         'location.country':'Country'}, inplace=True)
csv_file.dropna(inplace=True)
csv_file.reset_index(inplace=True)
csv_file.drop(columns=['index'], inplace=True)
csv_file.head()

,Name,Address,Latitude,Longitude,City,Country
0,Special $1 Coffee & Doughnut Cart,Broadway,40.724252,-73.997709,New York,United States
1,West Side Coffee Shop,323 Church St,40.720174,-74.003915,New York,United States
2,Gregorys Coffee,649 Broadway,40.726889,-73.995828,New York,United States
3,Coffee Wagon,Hudson and King,40.728231,-74.005783,New York,United States
4,Counter Culture Coffee NYC,376 Broome St,40.720490,-73.996236,New York,United States


##### Lastly: We visualize the data we have into a folium map and mark each venue's location with a marker that pops up the place's name

In [5]:
the_map = folium.Map(location=[latitude, longitude], zoom_start=15)
for name, lat, lng in zip(csv_file['Name'], csv_file['Latitude'], csv_file['Longitude']):
    folium.Marker(
        [lat, lng],
        radius=4,
        color='blue',
        fill = True,
        fill_color = 'blue',
        popup=name,
        fill_opacity = 0.6
    ).add_to(the_map)
the_map